### Intrdouction: 

#### I was doing this project in a Hackthon called DataFest. The DataFest is held at UCLA, and dataset was given by a learning paltform called CourseKata which teaching students how to learn statistics. The main topic of this Hackthon was to find the insights of how to improve student's engagment.

#### In my opinion, our team should look into the response dataset which has all the responds in each question from each student, according to this idea, I looked into the response dataset and found out there were a lot of students cannot get the points from some specific type of questions. To gain more insight, I tried to merge response, item, and page views dataset together to find out what type of question do students struggle the most and why they only try once or twice attempt for thse questions.

#### My jobs in my team was to merge dataset together, so the merged dataset could helped my teammates to do more analysis.

#### Due to the policy of the DataFest, I cannot show any information about the datasets from CourseKata, so only the codes are shown in this notebook.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import is_numeric_dtype
import re

sns.set_theme()

In [ ]:
%run -i "Feature_Engineering_Functions.py"

In [ ]:
responses = pd.read_csv("Data/Full Data/responses.csv")

In [ ]:
items = pd.read_csv("Data/Full Data/items.csv")

In [ ]:
page_views = pd.read_csv("Data/Full Data/page_views.csv")

## Response

In [ ]:
responses.head()

In [ ]:
responses.tail()

In [ ]:
responses.info()

In [ ]:
responses.describe()

In [ ]:
response_cols = list(responses.describe().columns)

In [ ]:
response_cols = ["chapter", "item_id", "item_type", "lrn_question_reference"] + response_cols + ["student_id", "page", "completes_page"]

In [ ]:
response_cols

In [ ]:
response_interested = responses[response_cols].copy()
response_interested.head()

In [ ]:
response_interested.tail()

In [ ]:
count_uniques(response_interested)

### Group by "chapter", "item_id", "item_type", "lrn_question_reference"

In [ ]:
response_interested.groupby(["chapter", "item_id", "item_type", "lrn_question_reference"]).apply(pd.DataFrame).drop(["chapter", "item_id", "lrn_question_reference", "item_type"], axis=1)

### Group by "chapter", "item_id", "item_type", "lrn_question_reference" sum points_possible

In [ ]:
totalPossiblePoints = response_interested.groupby(["chapter", "item_id", "item_type", "lrn_question_reference", "student_id"])["points_possible"].sum()
totalPossiblePoints

### Group by "chapter", "item_id", "item_type", "lrn_question_reference" sum points_earned

In [ ]:
totalEarnedPts = response_interested.groupby(["chapter", "item_id", "item_type", "lrn_question_reference", "student_id"])["points_earned"].sum()
totalEarnedPts

The last 5 rows of the dataframe showed some students cannot even earned points from a specific question, so we decided to move forward to inveestigate the why students cannot get pionts from these questions

I also try to incorporate the nPiontsDataFrame with the number of attempt for these questions

In [ ]:
totalAttempts = response_interested.groupby(["chapter", "item_id", "item_type", "lrn_question_reference", "student_id"])[["attempt"]].sum()
totalAttempts_df = pd.DataFrame(totalAttempts).reset_index()

In [ ]:
totalAttempts_df

### Attempts and points earned for each studnet

In [ ]:
attempts_ptEarned = response_interested.groupby(["chapter", "item_id", "item_type", 
                                                 "lrn_question_reference", "student_id", "page"])[["attempt", "points_earned"]].sum()
attempts_ptEarned = attempts_ptEarned.reset_index()
attempts_ptEarned

### Divide points_earned by points_possible and get the dataframe that are the questions that students cannot complete

In [ ]:
proportionEarned = totalEarnedPts / totalPossiblePoints
noPoint_indices = np.where(pd.to_numeric(proportionEarned, errors='coerce').notnull() == False)[0]
noPoints_info = proportionEarned[noPoint_indices]

In [ ]:
noPoints_df = pd.DataFrame(noPoints_info).reset_index()
noPoints_df = noPoints_df.iloc[:, 0:5]
noPoints_df

### Select no pionts from attempts and earned pionts data frame

In [ ]:
attempts_noPts = attempts_ptEarned.iloc[noPoint_indices, :].copy().reset_index(drop=True)
attempts_noPts

#### Most Student at least try it once

In [ ]:
any(attempts_noPts["attempt"] == 0)

### countplot for item type.

The item type is the type of question.

In [ ]:
sns.countplot(attempts_noPts["item_type"])

### Completeness of each chapter

In [ ]:
response_interested.groupby(["chapter"])["completes_page"].mean().sort_values()

I used the completes_page varibale to find out the completeness of each chapter. The result is that the cha12 has the lowest completeness percentage.

### Extract section, question, Question, and drop the item_df that does not have these elements

I tried to split the item_id into three different columns, so we can clearly see what section of the question is, what topic of the qustion is, and what question number is it.

In [ ]:
splited_ids = attempts_noPts["item_id"].str.split("_")
chTitle_ques = [i for i in splited_ids if len(i) == 3]
true_3 = [len(i) == 3 for i in splited_ids]
indciesOf_3 = np.where(true_3)[0]

In [ ]:
title_Ch = [i[0].startswith("Ch") for i in chTitle_ques]
title_ch_indices = np.where(title_Ch)[0]

title_noCh = [not i for i in title_Ch]
title_notCh_indices = np.where(title_noCh)[0]

In [ ]:
chTitle_ques_df = pd.DataFrame(chTitle_ques)

title_notCh_df = chTitle_ques_df.iloc[title_notCh_indices, :]
goodTitle = title_notCh_df[0] + "-" + title_notCh_df[1]

In [ ]:
chTitle_ques_df.columns = ["Section", "Question", "Number"]

In [ ]:
ch_ques = chTitle_ques_df.iloc[title_ch_indices, :]
nonCh_ques = chTitle_ques_df.iloc[title_notCh_indices, :]

In [ ]:
ch_ques.columns = ["Section", "Question", "Number"]
ch_ques

### Modify the StudBack value

In [ ]:
nonCh_ques.columns = ["Section", "Question", "Number"]
nonCh_ques

To make more value more general, I tired to find their common name. For exmaple, the question topics are StudBack1 and StudBack2, so I could simplfy them into just StudBack

In [ ]:
indices_studBack2 = nonCh_ques["Question"].str.contains("StudBack2")
indices_studBack2 = np.where(indices_studBack2)[0]

indices_studBack1 = nonCh_ques["Question"].str.contains("StudBack1")
indices_studBack1 = np.where(indices_studBack1)[0]

nonCh_ques.iloc[indices_studBack1, 1:2] = "StudBack"
nonCh_ques.iloc[indices_studBack2, 1:2] = "StudBack"

In [ ]:
nonCh_ques

In [ ]:
# columns = ["Ch", "Question Type"]
# # Create a DataFrame
# noPts_df = pd.DataFrame(chTitle_ques, columns=columns)
# noPts_df.to_csv("noPtsQuestions.csv", index=False)

### Extract values from the chTitle_ques

In [ ]:
chTitle_ques_sp = []
for i in chTitle_ques:
    for j in i:
        chTitle_ques_sp.append(j)

In [ ]:
chTitle_ques_df

In [ ]:
sns.barplot(noPoints_df["chapter"].value_counts(), orient="h")
plt.show()

## Grouping all types of Questions together and assgin them a reference

In [ ]:
noPoints_item3_df = attempts_noPts.copy()
noPoints_item3_df = noPoints_item3_df.iloc[indciesOf_3, :]

In [ ]:
noPoints_item3_df

In [ ]:
itemID = noPoints_item3_df["item_id"].str.split("_")
section = []
question = []
question_num = []

for i in itemID:
    section.append(i[0])
    question.append(i[1])
    question_num.append(i[2])

In [ ]:
itemID_df = pd.DataFrame(columns=["Section", "Question", "Question_Numer"])
itemID_df["Section"] = section
itemID_df["Question"] = question
itemID_df["Question_Numer"] = question_num

In [ ]:
itemID_df

In [ ]:
noPoints_item3_spDf = noPoints_item3_df.copy()

In [ ]:
noPoints_item3_spDf["Section"] = section
noPoints_item3_spDf["Question"] = question
noPoints_item3_spDf["Question_Numer"] = question_num

In [ ]:
noPoints_item3_spDf

In [ ]:
# noPoints_total_df.to_csv("noPts_total.csv")

### Attempts for each Question

In [ ]:
totalAttempts_specificQuestions = response_interested.groupby(["item_id", "item_type", "lrn_question_reference"])["attempt"].sum()
totalAttempts_specificQuestions.sort_values(ascending=False)[0:20]

In [ ]:
totalAttempts = response_interested.groupby(["item_id", "item_type"])["attempt"].sum()
totalAttempts.sort_values(ascending=False)[0:20]
noPts_totalAttempts = pd.DataFrame(totalAttempts).reset_index()
noPts_totalAttempts

In [ ]:
totalAttempts_uniqueQ = response_interested.groupby(["item_id", "item_type"])["attempt"].sum()
totalAttempts_uniqueQ.sort_values(ascending=False)[0:20]
totalAttempts_uniqueQ_df = pd.DataFrame(totalAttempts_uniqueQ).reset_index()
totalAttempts_uniqueQ_df

In [ ]:
sns.countplot(noPts_totalAttempts["item_type"])

In [ ]:
totalAttempts_specificQuestions_df = pd.DataFrame(totalAttempts_specificQuestions).reset_index()
sns.countplot(totalAttempts_specificQuestions_df.item_type)

In [ ]:
responses[["chapter","response", "prompt"]]

## Items

In this section, I merged noPionts dataset with the item on lrn_question_reference because I wanted to extract lrn_type which is the question type and the review falg which is if the question is the review section

The reason why I wanted to merge and got lrn_type and review falg variables because we may can find something interesting to know what type of questions that student cannot get the points.

In [ ]:
len(items)

In [ ]:
items.info()

In [ ]:
items.head()

In [ ]:
items_interested = items[["lrn_question_reference", "lrn_type", "review_flag"]].copy()

In [ ]:
items.columns

In [ ]:
responses.columns

In [ ]:
responses.completes_page

### Merging (noPionts dataset become noPts_respon_item_df)

In [ ]:
noPts_respon_item_df = pd.merge(noPoints_item3_spDf, items_interested, how='left',on="lrn_question_reference")
noPts_respon_item_df

In [ ]:
sns.barplot(noPts_respon_item_df["review_flag"].value_counts())

Most questions that the students cannot get the pionts do not come from the review section

In [ ]:
# respon_item_df.to_csv("respon_item_df.csv")

## Page Views

In this section, I wanted to merge the page views and noPts_respon_item_df on page and student id to see how many time they work on these assginments but still cannot get the pionts based on the engaged variables

In [ ]:
page_views.head()

In [ ]:
page_views.info()

In [ ]:
page_views_interested = page_views[["student_id", "page", "engaged"]].copy()
page_views_interested

In [ ]:
noPts_respon_item_df.head()

In [ ]:
timeOnPage = pd.DataFrame(page_views_interested.groupby(["student_id", "page"])["engaged"].sum()).reset_index()
timeOnPage

### noPts_respon_item_pgview_engaged_df

In [ ]:
noPts_respon_item_pgview_engaged_df = noPts_respon_item_df.merge(timeOnPage, on=["student_id", "page"])
noPts_respon_item_pgview_engaged_df

In [ ]:
# noPts_respon_item_pgview_engaged_df.to_csv("noPts_respon_item_pgview_engaged_df.csv")